In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensor

import torch.nn.functional as F
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

In [2]:
# reproduce
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(42)

In [3]:
# train.csv | train_mixup.csv | train_cutmix.csv | train_seq.csv
df = pd.read_csv("../input/train_seq.csv")
print(f"df shape is {df.shape}")
df.head()

df shape is (131, 2)


,img_name,target
0,RT3_1,1
1,RT3_2,1
2,RT3_3,1
3,RT3_4,1
4,RT3_5,1


In [4]:
IMAGE_SIZE = 224
class UmdDataset(Dataset):
    def __init__(self, df: pd.DataFrame, imfolder: str, transforms=None):
        self.df = df
        self.imfolder = imfolder
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        im_path = os.path.join(self.imfolder, self.df.iloc[index]['img_name'] + '.png')
        x = cv2.imread(im_path, 0)
        x = cv2.cvtColor(x, cv2.COLOR_GRAY2RGB)
        
        if self.transforms:
            augmented = self.transforms(image=x)
            x = augmented['image']
            
        y = self.df.loc[index]['target']
        return x, y
    
df_transform = A.Compose([
    A.RandomCrop(IMAGE_SIZE, IMAGE_SIZE, p=1),
    A.Normalize(),
    ToTensor(),
])

In [5]:
training_data_path = "../input/"
umd_dataset = UmdDataset(df=df, imfolder=training_data_path, transforms=df_transform)
train_loader = DataLoader(dataset=umd_dataset, batch_size=1, num_workers=16, shuffle=True)

In [6]:
# # visualize images
# for images, labels in train_loader:  
#     img = images.squeeze(0).permute((1, 2,0))
#     plt.imshow(img)
#     plt.show()

In [7]:
# # alexnet
# umd_features = []
# umd_y = []

# model = models.alexnet(pretrained=True)

# for epoch in tqdm(range(10)):
    
#     for images, labels in train_loader:  

# #         features = model.extract_features(images)
# #         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
#         features = model.features(images)
#         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).cpu().detach().numpy()
#         features = features.reshape([-1])
#         umd_features.append(features)
# #         umd_features.append(features.cpu().detach().numpy())
#         umd_y.append(labels.detach().numpy())

# umd_features = np.array(umd_features)
# umd_y = np. array(umd_y)

In [8]:
umd_features = []
umd_y = []

model = models.resnet18(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()

for epoch in tqdm(range(10)):
    
    for images, labels in train_loader:  

#         features = model.extract_features(images)
#         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
        features = feature_extractor(images)
        features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
#         features = features.reshape([-1])
        umd_features.append(features)
#         umd_features.append(features.cpu().detach().numpy())
        umd_y.append(labels.detach().numpy())

umd_features = np.array(umd_features)
umd_y = np. array(umd_y)

100%|██████████| 10/10 [00:27<00:00,  2.78s/it]


In [9]:
print(f"image feature shape is {umd_features.shape}")
print(f"image label shape is {umd_y.shape}")

image feature shape is (1310, 512)
image label shape is (1310, 1)


In [10]:
df_umd = pd.DataFrame(umd_features)
df_umd["target"] = umd_y

print(df_umd.shape)
df_umd.head()

(1310, 513)


,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,target
0,0.116135,0.788059,2.428497,1.056603,0.820298,0.141416,0.585964,0.087065,2.002062,1.639428,...,0.113385,0.108759,1.317424,0.147004,0.091513,0.761785,0.021018,0.497069,0.651833,2
1,0.036710,1.162364,0.306220,2.057626,0.178215,0.004166,0.418245,0.032669,0.616907,0.184875,...,0.293708,0.000000,1.341466,0.084182,0.009431,0.078457,0.427631,0.401455,0.059122,2
2,0.652957,0.981740,3.545043,0.204184,1.654799,0.000000,0.192026,0.052484,3.965159,0.283542,...,0.046583,0.000000,1.242455,0.682789,2.694038,0.355010,0.266838,0.000000,0.031524,2
3,0.000000,0.125300,0.359766,0.495549,0.193161,0.004342,0.537551,0.110779,0.913090,1.682503,...,0.238403,0.046362,0.988084,0.412065,0.046384,0.884869,0.007294,0.970891,0.072586,1
4,0.066916,0.402810,0.377425,0.022838,0.020259,0.184069,0.024513,0.412781,4.945152,0.332384,...,0.191347,0.129359,1.783438,0.626252,0.061745,0.324317,1.037723,0.002276,0.164416,3


In [11]:
df_umd_1 = df_umd[df_umd.target == 1]
df_umd_2 = df_umd[df_umd.target == 2]
df_umd_3 = df_umd[df_umd.target == 3]

In [12]:
train = pd.concat([df_umd_1[:200], df_umd_2[:200]], axis=0)
test = pd.concat([df_umd_1[200:], df_umd_2[200:], df_umd_3[:200]], axis=0)

train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)

print(f"train shape is {train.shape}, test shape is {test.shape}")

train shape is (400, 513), test shape is (600, 513)


In [13]:
df_umd_12 = df_umd[df_umd.target == 12].reset_index(drop=True) # merged class 1&2 image

# train.to_csv("../features/train_features.csv", index=False)
# test.to_csv("../features/test_features.csv", index=False)
# df_umd_12.to_csv("../features/cutmix_seq_features.csv", index=False)

In [14]:
# ### below is to generate transition sequences

# cutmix_seq_dataset = UmdDataset(df=df[120:].reset_index(), imfolder=training_data_path, transforms=df_transform)
# cutmix_seq_loader = DataLoader(dataset=cutmix_seq_dataset, batch_size=1, num_workers=1, shuffle=False)

# umd_features = []
# umd_y = []

# model = models.resnet18(pretrained=True)
# feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
# feature_extractor.eval()

# for epoch in tqdm(range(1)):
    
#     for images, labels in cutmix_seq_loader:  

# #         features = model.extract_features(images)
# #         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
#         features = feature_extractor(images)
#         features = nn.AdaptiveAvgPool2d(1)(features).squeeze(0).squeeze(1).squeeze(1).cpu().detach().numpy()
# #         features = features.reshape([-1])
#         umd_features.append(features)
# #         umd_features.append(features.cpu().detach().numpy())
#         umd_y.append(labels.detach().numpy())

# umd_features = np.array(umd_features)
# umd_y = np. array(umd_y)

# df_umd_12 = pd.DataFrame(umd_features)
# df_umd_12["target"] = umd_y

# print(df_umd_12.shape)
# df_umd_12.head()
# # df_umd_12.to_csv("../features/cutmix_seq_features.csv", index=False)

100%|██████████| 1/1 [00:00<00:00,  3.85it/s]

(11, 513)
